In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
from pathlib import Path 
from utils.numpy_encode import *
from dataloader import *
from learner import *
from config import *
import torch
from fastai.callbacks import SaveModelCallback
torch.__version__

Using cache found in C:\Users\TANFR/.cache\torch\hub\pytorch_vision_v0.4.2


'1.2.0'

In [2]:
# Location of your midi filesfiles
dataset = "mirex_labelled"
midi_path = Path(f'../dataset/midi/{dataset}')
midi_path.mkdir(parents=True, exist_ok=True)

# Location to save dataset
data_path = Path(f'../dataset/processed_transformer/')
data_path.mkdir(parents=True, exist_ok=True)

data_save_name = f'{dataset}_data_save.pkl'

In [3]:
midi_files = get_files(midi_path, '.mid', recurse=True)
print(len(midi_files))

196


## 3. Load Model

In [6]:
batch_size = 8
encode_position = True
dl_tfms = [batch_position_tfm] if encode_position else []
data = load_data(data_path, data_save_name, bs=batch_size, encode_position=encode_position, dl_tfms=dl_tfms)
data.get(0).__dict__

batch size:  (8, 71, 3)
batch size:  (8, 71, 3)
batch size:  (8, 71, 3)


C:\ProgramData\Anaconda3\envs\toy\lib\site-packages\torch\serialization.py:453: SourceChangeWarning: source code of class 'torch.nn.modules.loss.CrossEntropyLoss' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


{'data': array([  0,   1,   8, 185, ...,  59, 143,  52, 141]),
 'vocab': <utils.vocab.MusicVocab at 0x2d9f9084240>,
 '_stream': None,
 '_position': array([  0,   0,   0,   0, ..., 844, 844, 844, 844]),
 '_emotion': 4,
 '_midi_file': WindowsPath('../dataset/midi/mirex_labelled/amusement_549.mid')}

In [7]:
config = default_config()
config['encode_position'] = encode_position
learn = music_model_learner(data, config=config.copy())

## 4. Train

In [6]:
# https://forums.fast.ai/t/how-to-make-early-stopping-in-fastai/48493/8

save_name = "default_model2"
callbacks = [SaveModelCallback(learn, every='improvement', monitor='valid_loss', name=save_name)]
learn.fit_one_cycle(100, callbacks=callbacks)

epoch,train_loss,valid_loss,accuracy,time
0,3.161260,3.580204,0.275311,10:34
1,2.605873,2.683931,0.375849,11:04
2,2.252592,2.483538,0.369537,11:10
3,2.106822,2.321141,0.383590,11:06
4,1.993669,2.201742,0.423340,11:13
5,1.795074,2.072007,0.436918,11:12
6,1.699959,1.947729,0.474611,11:06
7,1.653325,1.877909,0.485585,11:08
8,1.377465,1.727233,0.513865,11:17
9,1.452904,1.800635,0.520268,11:13


Better model found at epoch 0 with valid_loss value: 3.5802040100097656.
Better model found at epoch 1 with valid_loss value: 2.6839308738708496.
Better model found at epoch 2 with valid_loss value: 2.4835381507873535.
Better model found at epoch 3 with valid_loss value: 2.321141242980957.
Better model found at epoch 4 with valid_loss value: 2.20174241065979.
Better model found at epoch 5 with valid_loss value: 2.072006940841675.
Better model found at epoch 6 with valid_loss value: 1.9477286338806152.
Better model found at epoch 7 with valid_loss value: 1.8779088258743286.
Better model found at epoch 8 with valid_loss value: 1.727232575416565.
Better model found at epoch 11 with valid_loss value: 1.5600841045379639.
Better model found at epoch 12 with valid_loss value: 1.4724549055099487.
Better model found at epoch 13 with valid_loss value: 1.4038087129592896.
Better model found at epoch 14 with valid_loss value: 1.3961058855056763.
Better model found at epoch 15 with valid_loss value

KeyboardInterrupt: 

### Start Predictions:

In [8]:
model_name = "default_model2"
data = MusicDataBunch.empty('../dataset/processed_transformer/')
vocab = data.vocab
model = music_model_learner(data=data, config=config, pretrained_path=f'../dataset/processed_transformer/models/{model_name}.pth')

batch size:  (64, 71, 3)
batch size:  (64, 71, 3)
batch size:  (64, 71, 3)


In [11]:
for i in [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]:
    image_path = f"../dataset/images/{i}.jpg"
    pred, _ = model.predict_from_image(image_path=image_path, n_words=1000)
    output_name = f'{i}.MID'
    stream = pred.to_stream()
    stream.write('midi', fp=output_name)

feature_map torch.Size([1, 512, 1])


feature_map torch.Size([1, 512, 1])


feature_map torch.Size([1, 512, 1])


feature_map torch.Size([1, 512, 1])


feature_map torch.Size([1, 512, 1])


...

feature_map torch.Size([1, 512, 1])


feature_map torch.Size([1, 512, 1])


feature_map torch.Size([1, 512, 1])


feature_map torch.Size([1, 512, 1])


feature_map torch.Size([1, 512, 1])


In [10]:
output_name = '1.MID'
stream = pred.to_stream()
stream.write('midi', fp=output_name)

'1.MID'